In [35]:
import glob
import torch
import json
import os
import pandas as pd 
from detectron2.evaluation.coco_evaluation import instances_to_coco_json

In [2]:
data_dir = "../data/rockfish_full/"

In [3]:
pths = glob.glob(data_dir + "**/*.pth", recursive=True)
len(pths)

145

In [4]:
pths

['../data/rockfish_full/RL-16_06/d20161027_7/stbd/stbd_rectified/frcnn-r101-pct100_preds.pth',
 '../data/rockfish_full/RL-16_06/d20161027_7/stbd/stbd_rectified/frcnn-r101_preds.pth',
 '../data/rockfish_full/RL-16_06/d20161027_7/stbd/stbd_rectified/frcnn-r101-pct25_preds.pth',
 '../data/rockfish_full/RL-16_06/d20161027_7/port/port_rectified/frcnn-r101-pct100_preds.pth',
 '../data/rockfish_full/RL-16_06/d20161027_7/port/port_rectified/frcnn-r101_preds.pth',
 '../data/rockfish_full/RL-16_06/d20161027_7/port/port_rectified/frcnn-r101-pct25_preds.pth',
 '../data/rockfish_full/RL-16_06/d20161028_8/stbd/stbd_rectified/frcnn-r101-pct100_preds.pth',
 '../data/rockfish_full/RL-16_06/d20161028_8/stbd/stbd_rectified/frcnn-r101_preds.pth',
 '../data/rockfish_full/RL-16_06/d20161028_8/stbd/stbd_rectified/frcnn-r101-pct25_preds.pth',
 '../data/rockfish_full/RL-16_06/d20161028_8/port/port_rectified/frcnn-r101-pct100_preds.pth',
 '../data/rockfish_full/RL-16_06/d20161028_8/port/port_rectified/frcnn-r10

In [9]:
anno_id = 0
for pth in pths:
    loaded = torch.load(pth)
    
    ims = []
    annos = []

    for im in loaded:
        ims.append({
                'file_name': im['file_name'],
                'id': im['file_name'],
                'height': im["instances"]._image_size[0],
                'width': im["instances"]._image_size[1]
            })

        instances = instances_to_coco_json(im['instances'], im['file_name'])
        annos += instances
    
    for i, anno in enumerate(annos):
        anno['id'] = anno_id
        anno['category_id'] = 1 # need to map to our categories
        anno_id += 1

    print(pth, "ims", len(ims))
    print(pth, "annos", len(annos))
    
    full_coco = { 'info': {},
                 'licenses': [],
                 'categories': [ { "supercategory": "THING", "id": 1 , "name": "THING" }],
                 'images': ims,
                 'annotations': annos
                }
    
    output_fname = pth[:-4] + "_coco.json"
    json.dump(full_coco, open(output_fname, "w"))

../data/rockfish_full/RL-16_06/d20161027_7/stbd/stbd_rectified/frcnn-r101-pct100_preds.pth ims 1082
../data/rockfish_full/RL-16_06/d20161027_7/stbd/stbd_rectified/frcnn-r101-pct100_preds.pth annos 6020
../data/rockfish_full/RL-16_06/d20161027_7/stbd/stbd_rectified/frcnn-r101_preds.pth ims 1082
../data/rockfish_full/RL-16_06/d20161027_7/stbd/stbd_rectified/frcnn-r101_preds.pth annos 52365
../data/rockfish_full/RL-16_06/d20161027_7/stbd/stbd_rectified/frcnn-r101-pct25_preds.pth ims 1082
../data/rockfish_full/RL-16_06/d20161027_7/stbd/stbd_rectified/frcnn-r101-pct25_preds.pth annos 1679
../data/rockfish_full/RL-16_06/d20161027_7/port/port_rectified/frcnn-r101-pct100_preds.pth ims 1082
../data/rockfish_full/RL-16_06/d20161027_7/port/port_rectified/frcnn-r101-pct100_preds.pth annos 6055
../data/rockfish_full/RL-16_06/d20161027_7/port/port_rectified/frcnn-r101_preds.pth ims 1082
../data/rockfish_full/RL-16_06/d20161027_7/port/port_rectified/frcnn-r101_preds.pth annos 48867
../data/rockfish_f

## Remove validation set from self-train coco files

In [13]:
val_json_path = "../data/rockfish_labels/coco/jimmy_lynker_val_5k_25k_v2.json"

with open(val_json_path, 'r') as f:
    val_json = json.load(f)

In [6]:
coco_paths = [pth[:-4] + "_coco.json" for pth in pths]
coco_paths[0]

'../data/rockfish_full/RL-16_06/d20161027_7/stbd/stbd_rectified/frcnn-r101-pct100_preds_coco.json'

In [11]:
[c['file_name'] for c in coco['images']]

['../data/rockfish_full/RL-16_06/d20161027_7/stbd/stbd_rectified/20161027.174138.00060_rect_color.jpg',
 '../data/rockfish_full/RL-16_06/d20161027_7/stbd/stbd_rectified/20161027.174146.00063_rect_color.jpg',
 '../data/rockfish_full/RL-16_06/d20161027_7/stbd/stbd_rectified/20161027.174154.00066_rect_color.jpg',
 '../data/rockfish_full/RL-16_06/d20161027_7/stbd/stbd_rectified/20161027.174202.00069_rect_color.jpg',
 '../data/rockfish_full/RL-16_06/d20161027_7/stbd/stbd_rectified/20161027.174210.00072_rect_color.jpg',
 '../data/rockfish_full/RL-16_06/d20161027_7/stbd/stbd_rectified/20161027.174218.00075_rect_color.jpg',
 '../data/rockfish_full/RL-16_06/d20161027_7/stbd/stbd_rectified/20161027.174226.00078_rect_color.jpg',
 '../data/rockfish_full/RL-16_06/d20161027_7/stbd/stbd_rectified/20161027.174234.00081_rect_color.jpg',
 '../data/rockfish_full/RL-16_06/d20161027_7/stbd/stbd_rectified/20161027.174242.00084_rect_color.jpg',
 '../data/rockfish_full/RL-16_06/d20161027_7/stbd/stbd_rectified

In [37]:
val_fns = [os.path.basename(v['file_name']) for v in val_json['images']]
val_fns

['20170819.185225.00691.tif',
 '20170819.192202.01357.tif',
 '20170819.192650.01465.tif',
 '20170819.193434.01639.tif',
 '20170819.195402.02077.tif',
 '20170819.202722.02827.tif',
 '20170819.203259.02953.tif',
 '20170819.204027.03121.tif',
 '20170819.210411.03655.tif',
 '20170819.210459.03673.tif',
 '20170819.211803.03967.tif',
 '20170819.211851.03985.tif',
 '20170819.214307.04531.tif',
 '20170819.215332.04765.tif',
 '20170819.221300.05203.tif',
 '20170820.172306.00190.tif',
 '20170820.172322.00196.tif',
 '20170820.172442.00226.tif',
 '20170820.173138.00382.tif',
 '20170820.173954.00568.tif',
 '20170820.174738.00742.tif',
 '20170820.175450.00904.tif',
 '20170820.181243.01306.tif',
 '20170820.182219.01522.tif',
 '20170820.194724.03436.tif',
 '20170820.203524.04516.tif',
 '20170820.205053.04864.tif',
 '20170820.205701.05002.tif',
 '20170820.211613.05434.tif',
 '20170820.214357.06058.tif',
 '20170820.222150.06910.tif',
 '20170820.222254.06934.tif',
 '20170820.222414.06964.tif',
 '20170820

In [22]:
len(val_fns)

1042

In [41]:
import tqdm

# for each coco:
for cp in tqdm.tqdm(coco_paths):
    # load the file
    with open(cp, 'r') as f: coco = json.load(f)
        
    # find images in coco that are also in val, call that c_v_intersect
    c_v_intersect = []
    for im in coco['images']:
        im_path = im['file_name'][8:] # remove ""../data/"
        fn = os.path.basename(im_path)
        if fn in val_fns:
            c_v_intersect.append(fn)

    # for each image in c_v_intersect
    for cvi in c_v_intersect:
        # remove the corresponding annotations for the image
        coco_annos_in_cvi = [c for c in coco['annotations']]
        print(len(coco_annos_in_cvi))
    # save coco

  0%|          | 0/145 [00:00<?, ?it/s]

on  ../data/rockfish_full/RL-16_06/d20161027_7/stbd/stbd_rectified/frcnn-r101-pct100_preds_coco.json
on  ../data/rockfish_full/RL-16_06/d20161027_7/stbd/stbd_rectified/frcnn-r101_preds_coco.json


  1%|▏         | 2/145 [00:00<00:32,  4.37it/s]

on  ../data/rockfish_full/RL-16_06/d20161027_7/stbd/stbd_rectified/frcnn-r101-pct25_preds_coco.json
on  ../data/rockfish_full/RL-16_06/d20161027_7/port/port_rectified/frcnn-r101-pct100_preds_coco.json
on  ../data/rockfish_full/RL-16_06/d20161027_7/port/port_rectified/frcnn-r101_preds_coco.json


  5%|▍         | 7/145 [00:00<00:22,  6.10it/s]

...

on  ../data/rockfish_full/SHE-11/d20111015_18/stbd/uncropped_corrected/frcnn-r101-pct25_preds_coco.json
on  ../data/rockfish_full/SHE-11/d20111015_18/port/uncropped_corrected/frcnn-r101-pct100_preds_coco.json
14432
14432
14432
14432
14432
14432
14432
14432
14432
14432
14432
14432
14432
on  ../data/rockfish_full/SHE-11/d20111015_18/port/uncropped_corrected/frcnn-r101-pct25_preds_coco.json
2611
2611
2611
2611
2611
2611
2611
2611
2611
2611
2611
2611
2611
on  ../data/rockfish_full/SHE-11/d20111011_10/stbd/uncropped_corrected/frcnn-r101-pct100_preds_coco.json
on  ../data/rockfish_full/SHE-11/d20111011_10/stbd/uncropped_corrected/frcnn-r101-pct25_preds_coco.json
on  ../data/rockfish_full/SHE-11/d20111011_10/port/uncropped_corrected/frcnn-r101-pct100_preds_coco.json
on  ../data/rockfish_full/SHE-11/d20111011_10/port/uncropped_corrected/frcnn-r101-pct25_preds_coco.json
on  ../data/rockfish_full/SHE-11/d201110117_21/stbd/uncropped_corrected/frcnn-r101-pct100_preds_coco.json


 89%|████████▉ | 129/145 [00:12<00:00, 32.52it/s]

2977
2977
2977
2977
2977
2977
2977
2977
2977
2977
2977
2977
2977
2977
2977
2977
2977
2977
2977
2977
2977
2977
2977
2977
2977
2977
2977
2977
2977
2977
2977
2977
2977
2977
2977
2977
on  ../data/rockfish_full/SHE-11/d201110117_21/stbd/uncropped_corrected/frcnn-r101-pct25_preds_coco.json
802
802
802
802
802
802
802
802
802
802
802
802
802
802
802
802
802
802
802
802
802
802
802
802
802
802
802
802
802
802
802
802
802
802
802
802
on  ../data/rockfish_full/SHE-11/d201110117_21/port/uncropped_corrected/frcnn-r101-pct100_preds_coco.json
on  ../data/rockfish_full/SHE-11/d201110117_21/port/uncropped_corrected/frcnn-r101-pct25_preds_coco.json
on  ../data/rockfish_full/SHE-11/d20111007_2/stbd/uncropped_corrected/frcnn-r101-pct100_preds_coco.json
on  ../data/rockfish_full/SHE-11/d20111007_2/stbd/uncropped_corrected/frcnn-r101-pct25_preds_coco.json
on  ../data/rockfish_full/SHE-11/d20111007_2/port/uncropped_corrected/frcnn-r101-pct100_preds_coco.json
on  ../data/rockfish_full/SHE-11/d20111007_2/port

100%|██████████| 145/145 [00:12<00:00, 11.33it/s]

on  ../data/rockfish_full/SH-18-12/d20181014_2/port/port_rectified/frcnn-r101-pct25_preds_coco.json
on  ../data/rockfish_full/SH-18-12/d20181012_1/stbd/stbd_rectified/frcnn-r101-pct100_preds_coco.json
on  ../data/rockfish_full/SH-18-12/d20181012_1/stbd/stbd_rectified/frcnn-r101-pct25_preds_coco.json
on  ../data/rockfish_full/SH-18-12/d20181012_1/port/port_rectified/frcnn-r101-pct100_preds_coco.json
on  ../data/rockfish_full/SH-18-12/d20181012_1/port/port_rectified/frcnn-r101_preds_coco.json
on  ../data/rockfish_full/SH-18-12/d20181012_1/port/port_rectified/frcnn-r101-pct25_preds_coco.json
on  ../data/rockfish_full/SH-18-12/d20181015_3/port/port_rectified/frcnn-r101-pct100_preds_coco.json
on  ../data/rockfish_full/SH-18-12/d20181015_3/port/port_rectified/frcnn-r101_preds_coco.json
on  ../data/rockfish_full/SH-18-12/d20181015_3/port/port_rectified/frcnn-r101-pct25_preds_coco.json
